In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

# Profile Page

In [2]:
username_list=['henjiwong','hungryfever','cookingwithhel','yackikuka','devispantry','xanderskitchen']

In [3]:
# browser = webdriver.Chrome(ChromeDriverManager().install())
        
# browser.get('https://www.instagram.com/accounts/login/')

# emailInput = browser.find_elements_by_css_selector('form input')[0]
# passwordInput = browser.find_elements_by_css_selector('form input')[1]

# emailInput.send_keys('instagramusername')
# passwordInput.send_keys('password')
# passwordInput.send_keys(Keys.ENTER)
# time.sleep(2)

username='henjiwong'
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.instagram.com/'+username+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/oliviatanuwidjaja/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [5]:
all_post = []

for username in username_list:
    browser.get('https://www.instagram.com/'+username+'/?hl=en')
    Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    post = 'https://www.instagram.com/p/'
    post_links = []
    while len(post_links) < 50:
        links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
        for link in links:
            if post in link and link not in post_links:
                post_links.append(link)
                all_post.append(link)
        scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
        browser.execute_script(scroll_down)
        time.sleep(3)
    else:
        post_links[:50]

In [6]:
all_post

['https://www.instagram.com/p/CELL_kAAdb_/',
 'https://www.instagram.com/p/CEI8cMagKOb/',
 'https://www.instagram.com/p/CEInTOXgGak/',
 'https://www.instagram.com/p/CEGCeoalCmp/',
 'https://www.instagram.com/p/CEEKTLcgbrk/',
 'https://www.instagram.com/p/CEDdrH9gICS/',
 'https://www.instagram.com/p/CEB33EhFqyk/',
 'https://www.instagram.com/p/CEBn8rBF3zp/',
 'https://www.instagram.com/p/CEBInVblm8j/',
 'https://www.instagram.com/p/CEA5MEflnBJ/',
 'https://www.instagram.com/p/CD-lePZly4y/',
 'https://www.instagram.com/p/CD-ThlIlwQ3/',
 'https://www.instagram.com/p/CD7vdfUlPBn/',
 'https://www.instagram.com/p/CD5aUbGFVgu/',
 'https://www.instagram.com/p/CD5KeZOlUo3/',
 'https://www.instagram.com/p/CD3Sx1GFoEo/',
 'https://www.instagram.com/p/CD2zsy5FPx4/',
 'https://www.instagram.com/p/CD2l5crldPT/',
 'https://www.instagram.com/p/CD0QhwIlug6/',
 'https://www.instagram.com/p/CD0BFgulhFy/',
 'https://www.instagram.com/p/CDySZvilqfQ/',
 'https://www.instagram.com/p/CDxruLDFWjF/',
 'https://

In [7]:
def find_hashtags(comment):
    """
    Find hastags used in comment and return them
    Args:
    comment: Instagram comment text
    Returns:
    a list or individual hashtags if found in comment
    """
    hashtags = re.findall('#[A-Za-z]+', comment)
    if (len(hashtags) > 1) & (len(hashtags) != 1):
        return hashtags
    elif len(hashtags) == 1:
        return hashtags[0]
    else:
        return ""


def find_mentions(comment):
    """
    Find mentions used in comment and return them
    Args:
    comment: Instagram comment text
    Returns:
    a list or individual mentions if found in comment
    """
    mentions = re.findall('@[A-Za-z]+', comment)
    if (len(mentions) > 1) & (len(mentions) != 1):
        return mentions
    elif len(mentions) == 1:
        return mentions[0]
    else:
        return ""


def insta_link_details(url):
    """
    Take a post url and return post details
    Args:
    urls: a list of urls for Instagram posts 
    Returns:
    A list of dictionaries with details for each Instagram post, including link,
    post type, like/view count, age (when posted), and initial comment
    """

    browser.get(url)
    try:
        # This captures the standard like count.
        likes = browser.find_element_by_xpath(
            """/html/body/div[1]/section/main/div/div/article/
                div[3]/section[2]/div/div/button/span""").text.split()[0]
        post_type = 'photo'
    except:
        # This captures the like count for videos which is stored
        likes = browser.find_element_by_xpath(
            """/html/body/div[1]/section/main/div/div/article/
                div[3]/section[2]/div/span/span""").text.split()[0]
        post_type = 'video'
    age = browser.find_element_by_css_selector('a time').text
    posted_date = browser.find_element_by_css_selector('a time').get_attribute("title")
    posted_time = browser.find_element_by_css_selector('a time').get_attribute("datetime")
    username = browser.find_element_by_xpath(
        """/html/body/div[1]/section/main/div/div[1]/article/
        div[3]/div[1]/ul/div/li/div/div/div[2]/h2""").text
    comment = browser.find_element_by_xpath(
        """/html/body/div[1]/section/main/div/div[1]/article/
        div[3]/div[1]/ul/div/li/div/div/div[2]/span""").text
    
    
    hashtags = find_hashtags(comment)
    mentions = find_mentions(comment)
    post_details = {'link': url, 'username':username, 'type': post_type, 'likes_views': likes,
                    'age': age, 'post_date': posted_date, 'post_time': posted_time, 
                    'comment': comment, 'hashtags': hashtags, 'mentions': mentions}
    time.sleep(10)
    return post_details

In [8]:
# all_post1 = all_post[:2]

# all_post_details1 = [insta_link_details(url) for url in all_post1]
# food_dataframe1 = pd.DataFrame(all_post_details1)
# food_dataframe1

In [9]:
all_post1 = all_post[:50]

all_post_details1 = [insta_link_details(url) for url in all_post1]
food_dataframe1 = pd.DataFrame(all_post_details1)
food_dataframe1

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CELL_kAAdb_/,henjiwong,video,"54,958",1h,"Aug 22, 2020",2020-08-22T03:26:10.000Z,DESSERT BOX BY NAJLA CADBURY\n-\n-\nYukkk biki...,,
1,https://www.instagram.com/p/CEI8cMagKOb/,henjiwong,video,"16,651",22h,"Aug 21, 2020",2020-08-21T06:38:46.000Z,PANDAN SERIES!!\n-\n-\nWah yang suka wangi pan...,"[#BobaSatuIndonesia, #Tablesituation, #ontheta...",@bube
2,https://www.instagram.com/p/CEInTOXgGak/,henjiwong,video,"61,829",23h,"Aug 21, 2020",2020-08-21T05:28:04.000Z,MOCHI ANAK KOS\n-\n-\nSbnarnya mau kasi resep ...,"[#Tablesituation, #onthetableproject, #lovetoe...",
3,https://www.instagram.com/p/CEGCeoalCmp/,henjiwong,video,"233,467",13h,"Aug 21, 2020",2020-08-21T14:58:49.000Z,Japanese Souffle Cheesecake TANPA Cream Cheese...,,
4,https://www.instagram.com/p/CEEKTLcgbrk/,henjiwong,video,"47,487",2d,"Aug 19, 2020",2020-08-19T14:50:17.000Z,CEMILAN SEHAT DULU YUKK!!\n-\n-\nKali ini kita...,,@infosuperindo
5,https://www.instagram.com/p/CEDdrH9gICS/,henjiwong,video,"267,974",1d,"Aug 20, 2020",2020-08-20T13:46:45.000Z,CAKE JELITA 🌹🌹🌹\n-\n-\nNih yang lagi rame... k...,"[#Tablesituation, #onthetableproject, #lovetoe...",
6,https://www.instagram.com/p/CEB33EhFqyk/,henjiwong,video,"56,309",3d,"Aug 18, 2020",2020-08-18T15:56:27.000Z,KEPOIN SAUSNYA!!\n-\n-\nSeneng banget pas gue ...,#kepoinsaosnya,
7,https://www.instagram.com/p/CEBn8rBF3zp/,henjiwong,photo,"3,791",3d,"Aug 18, 2020",2020-08-18T14:32:33.000Z,SOMAY BLACK MENTAI\n-\n-\nUd perna makan somay...,"[#Tablesituation, #onthetableproject, #lovetoe...","[@somay, @cnlulaby]"
8,https://www.instagram.com/p/CEBInVblm8j/,henjiwong,photo,"3,793",3d,"Aug 18, 2020",2020-08-18T06:53:43.000Z,BOBA CROISSANT!!\n-\n-\nSiapa yang suka makan ...,"[#Tablesituation, #onthetableproject, #lovetoe...",@bobataindonesia
9,https://www.instagram.com/p/CEA5MEflnBJ/,henjiwong,video,"206,815",3d,"Aug 19, 2020",2020-08-19T01:47:04.000Z,SWEET POTATO BREAD\n-\n-\nUd pada coba buat Ro...,"[#Tablesituation, #onthetableproject, #lovetoe...",


In [10]:
all_post2 = all_post[50:100]

all_post_details2 = [insta_link_details(url) for url in all_post2]
food_dataframe2 = pd.DataFrame(all_post_details2)
food_dataframe2

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CDFrIt8FqzB/,henjiwong,video,"934,571",1w,"Aug 9, 2020",2020-08-09T11:13:34.000Z,DESSERT BOX TURKISH ala-ala\n-\n-\nVersi pelan...,"[#Tablesituation, #onthetableproject, #lovetoe...",
1,https://www.instagram.com/p/CDD13DQlCil/,henjiwong,photo,"3,397",3w,"Jul 25, 2020",2020-07-25T11:34:24.000Z,BURGER 🍔 🍔🍔\n-\n-\nNih kalau makan burger pali...,"[#australianbeef, #burger, #beefup, #burgertim...","[@holysmokesid, @trueaussieid]"
2,https://www.instagram.com/p/CDDGC9OlAM_/,henjiwong,video,"199,298",3w,"Jul 30, 2020",2020-07-30T07:57:15.000Z,UBE CHEESE PANDESAL\n-\n-\nNamanya perna dgr g...,"[#Tablesituation, #onthetableproject, #lovetoe...",
3,https://www.instagram.com/p/CDBPtyeFAuE/,henjiwong,video,"40,797",4w,"Jul 24, 2020",2020-07-24T15:20:15.000Z,Dalgona + Boba\n-\n-\nMenu terbaru LIMITED EDI...,"[#Tablesituation, #onthetableproject, #lovetoe...","[@xiboba, @mbventura, @kulogroup]"
4,https://www.instagram.com/p/CDAvmJYFZEQ/,henjiwong,photo,"3,692",4w,"Jul 24, 2020",2020-07-24T05:46:42.000Z,MURABAE BENTO\n-\n-\nSiapa yang makan harus pa...,"[#Tablesituation, #onthetableproject, #lovetoe...",@murabaebento
5,https://www.instagram.com/p/CDAhiQ2lyd8/,henjiwong,video,"146,152",3w,"Aug 1, 2020",2020-08-01T03:12:44.000Z,SARANG SEMUT KARAMEL\n-\n-\nKapan terakhir kal...,"[#Tablesituation, #onthetableproject, #lovetoe...",
6,https://www.instagram.com/p/CC-qoIblTDS/,henjiwong,photo,"2,739",4w,"Jul 23, 2020",2020-07-23T11:53:50.000Z,Ud stock cemilan belum buat temen nonton drako...,#TokopediaNyam,
7,https://www.instagram.com/p/CC-OBkBlmhG/,henjiwong,video,"84,501",4w,"Jul 23, 2020",2020-07-23T10:30:26.000Z,CHIFFON CAKE!!\n-\n-\nNgebayangin buat Chiffon...,"[#Tablesituation, #onthetableproject, #lovetoe...",@ohanarecipe
8,https://www.instagram.com/p/CC98BiBFyjL/,henjiwong,video,"191,526",3w,"Jul 25, 2020",2020-07-25T06:59:40.000Z,ROJUSONG ( ROTI KEJU GOCONG)\n-\n-\nNih yang s...,"[#Tablesituation, #onthetableproject, #lovetoe...",
9,https://www.instagram.com/p/CC8L1U-lEzg/,henjiwong,photo,"4,052",4w,"Jul 22, 2020",2020-07-22T14:21:44.000Z,SPICY BEEF KARUBI\n-\n-\n\nWaktunya makan udon...,"[#BanyakCara, #MarugameUdon]",@marugameudon


In [11]:
all_post3 = all_post[100:150]

all_post_details3 = [insta_link_details(url) for url in all_post3]
food_dataframe3 = pd.DataFrame(all_post_details3)
food_dataframe3

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CDoV03hnNZx/,hungryfever,video,"209,225",1w,"Aug 15, 2020",2020-08-15T02:10:18.000Z,Lagi weekend-an auto kepikiran Chatuchak Marke...,"[#HungryFever, #Chatuchak, #HungryFeverBangkok...",
1,https://www.instagram.com/p/CDnXV05nWBF/,hungryfever,photo,"2,437",2w,"Aug 8, 2020",2020-08-08T04:39:59.000Z,Se-gampang itu BBQ-an di rumah. Baru aja nyeto...,"[#HungryFever, #KomatkaMeat, #eatingfortheinst...",@komatka
2,https://www.instagram.com/p/CDnJP7OHJsm/,hungryfever,video,"256,037",4d,"Aug 17, 2020",2020-08-17T16:00:13.000Z,[ DESSERT BOX COKLAT LUMER ]\n.\nMinggu penuh ...,"[#HungryFever, #ChefHungryFever, #HungryFeverV...",
3,https://www.instagram.com/p/CDlzL0ZHix8/,hungryfever,video,"312,310",6d,"Aug 15, 2020",2020-08-15T13:09:18.000Z,Se-fruit trobek lagi di @dittotea 🐰🐰🐰🐰🐰\n.\n#H...,"[#HungryFever, #DittoTea, #HungryFeverBangkok,...",@dittotea
4,https://www.instagram.com/p/CDk1b5qHuhl/,hungryfever,video,"68,774",2w,"Aug 7, 2020",2020-08-07T09:37:58.000Z,[ LEMON CAKE ]\n.\nLEMON makes the world a bet...,"[#HungryFever, #PoppyEats, #HungryFeverVideo, ...",@poppyeats
5,https://www.instagram.com/p/CDkkYEanudq/,hungryfever,video,"327,805",1w,"Aug 12, 2020",2020-08-12T02:37:38.000Z,[ BAKSO MANGKOK BERANAK ]\n.\nSetelah berkutat...,"[#HungryFever, #ChefHungryFever, #HungryFeverV...",
6,https://www.instagram.com/p/CDjMfWpBLCZ/,hungryfever,video,"34,031",2w,"Aug 6, 2020",2020-08-06T14:52:33.000Z,"The best French Gelato around Kuta, Bali @icel...","[#HungryFever, #IceLabBali, #HungryFeverBali, ...",@icelab
7,https://www.instagram.com/p/CDiNHT2HNw_/,hungryfever,video,"47,873",2w,"Aug 6, 2020",2020-08-06T05:47:35.000Z,Another approval stamp for PORTUGUESE EGG TART...,"[#HungryFever, #MaisumID, #HungryFeverVideo, #...",@maisum
8,https://www.instagram.com/p/CDh_1IynsTu/,hungryfever,video,"188,144",1w,"Aug 13, 2020",2020-08-13T16:27:43.000Z,[ PANCAKE OREO CREAM CHEESE ]\n- byRequest #8\...,"[#HungryFever, #ChefHungryFever, #HungryFeverV...",
9,https://www.instagram.com/p/CDglJ1sn_6y/,hungryfever,video,"66,745",2w,"Aug 5, 2020",2020-08-05T13:29:25.000Z,Cold Brew + Salted Foam @starbucksindonesia\n....,"[#HungryFever, #StarbucksIndonesia, #sipsofcoo...",@starbucksindonesia


In [12]:
all_post4 = all_post[150:200]

all_post_details4 = [insta_link_details(url) for url in all_post4]
food_dataframe4 = pd.DataFrame(all_post_details4)
food_dataframe4

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CDP_ogPFsGM/,cookingwithhel,video,"210,445",3w,"Jul 30, 2020",2020-07-30T04:23:41.000Z,"Es krim strawberry kesukaan anak-anak, kali in...","[#resepeskrim, #resepeskrimhomemade, #eskrimst...",
1,https://www.instagram.com/p/CDOE_saFbHK/,cookingwithhel,video,"229,808",2w,"Aug 5, 2020",2020-08-05T09:18:45.000Z,"Hai kali ini hel buat camilan kesukaan anak2, ...","[#FortuneCerMat, #PilihanIbuCerMat, #IbuCerMat...","[@fortunecermat, @fortunecermat, @fortunecerma..."
2,https://www.instagram.com/p/CDOExZGFOeL/,cookingwithhel,photo,"8,841",3w,"Jul 31, 2020",2020-07-31T15:34:55.000Z,"Hai kali ini hel buat camilan kesukaan anak2, ...","[#FortuneCerMat, #PilihanIbuCerMat, #IbuCerMat...","[@fortunecermat, @fortunecermat, @fortunecerma..."
3,https://www.instagram.com/p/CDNc1nTFlrF/,cookingwithhel,video,"364,118",3w,"Aug 1, 2020",2020-08-01T03:08:34.000Z,Salah satu snack favorit anak-anak stik kentan...,"[#stikkentang, #stikkentangkeju, #resepstikken...",
4,https://www.instagram.com/p/CDNKGbuFa8w/,cookingwithhel,video,"303,923",2w,"Aug 7, 2020",2020-08-07T12:04:05.000Z,"Unbaked strawberry cheesecake lumer, untuk res...","[#dessertbox, #strawberrycheesecakedessertbox,...",
5,https://www.instagram.com/p/CDLqyTRF5oF/,cookingwithhel,photo,"19,405",2w,"Aug 8, 2020",2020-08-08T02:16:51.000Z,Unbaked strawberry cheesecake lumer dessert bo...,"[#strawberrycheesecake, #unbakedstrawberrychee...",@cookingwithhel
6,https://www.instagram.com/p/CDKzq27lI8c/,cookingwithhel,video,"168,135",3w,"Jul 30, 2020",2020-07-30T04:37:01.000Z,"Strawberry cheeeecake lumer, Resep dan full vi...",,
7,https://www.instagram.com/p/CDIFcf4lgq4/,cookingwithhel,video,"353,247",3w,"Aug 1, 2020",2020-07-31T18:32:29.000Z,Milk bath cake versi ekonomis. Resep dan penje...,"[#milkbathcake, #milkbathcakeekonomis, #resepm...",
8,https://www.instagram.com/p/CDF0hg8FuD1/,cookingwithhel,photo,"11,762",3w,"Jul 31, 2020",2020-07-31T12:46:09.000Z,Milk bath cake dessert box ekonomis versi anak...,"[#milkbathcake, #dessertbox, #tresleches, #bol...",@cookingwithhel
9,https://www.instagram.com/p/CDDVAR8FlHh/,cookingwithhel,video,"160,172",3w,"Jul 26, 2020",2020-07-26T06:35:36.000Z,Resep dan video tutorial udah di upload di you...,"[#milkbathcake, #resepmilkbathcake]",


In [13]:
all_post5 = all_post[200:250]

all_post_details5 = [insta_link_details(url) for url in all_post5]
food_dataframe5 = pd.DataFrame(all_post_details5)
food_dataframe5

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CDWG8FJJhwv/,yackikuka,photo,"13,123",2w,"Aug 2, 2020",2020-08-02T03:57:35.000Z,MIE GORENG JAWA * JAVANESE FRIED NOODLE\n#masa...,"[#masakgapakeribet, #viral, #miegorengjawa, #h...",
1,https://www.instagram.com/p/CDTjbscJgHt/,yackikuka,video,"567,014",2w,"Aug 6, 2020",2020-08-06T03:13:17.000Z,CHICKEN CORDON BLEU with LEMON BUTTER SAUCE\n#...,"[#masakgapakeribet, #tgif, #homemade, #chicken...",@ademsari
2,https://www.instagram.com/p/CDQLdtpp80C/,yackikuka,photo,"16,442",3w,"Jul 31, 2020",2020-07-31T06:43:54.000Z,CHICKEN CORDON BLEU with LEMON BUTTER SAUCE\n#...,"[#masakgapakeribet, #happythursday, #homemade,...",@ademsari
3,https://www.instagram.com/p/CDOU9DvJcwF/,yackikuka,photo,"3,969",3w,"Jul 30, 2020",2020-07-29T20:23:59.000Z,Hai hai! Idul Adha tinggal beberapa hari lagi ...,"[#MasakBersamaSasaBumbuKomplit, #KreasiSasa]",
4,https://www.instagram.com/p/CDNpF7ppn6y/,yackikuka,video,"1,700,566",2d,"Aug 20, 2020",2020-08-19T19:09:42.000Z,SIMPLE SPAGHETTI BRULEE❤️\n#masakgapakeribet\n...,"[#masakgapakeribet, #happywednesday, #homemade...",
5,https://www.instagram.com/p/CDLdEg-J9lJ/,yackikuka,photo,"32,325",2w,"Aug 6, 2020",2020-08-06T09:28:54.000Z,SPAGHETTI BRULEE❤️\n#masakgapakeribet\n\nHappy...,"[#masakgapakeribet, #kekinian, #idejualan, #ha...",
6,https://www.instagram.com/p/CDIlaJvpLKr/,yackikuka,video,"382,537",2w,"Aug 4, 2020",2020-08-04T04:09:56.000Z,POPSICLE YOGURT\n#masakgapakeribet\n\nHappy Mo...,"[#masakgapakeribet, #kidsactivities, #Yogurtta...",@cimoryindonesia
7,https://www.instagram.com/p/CDGuobCpE2V/,yackikuka,photo,"4,355",3w,"Jul 27, 2020",2020-07-26T17:18:16.000Z,Bingung belanja buat masak or cari camilan?\nS...,#TokopediaNyam,
8,https://www.instagram.com/p/CDDuDKDpL1g/,yackikuka,photo,"22,327",3w,"Jul 29, 2020",2020-07-29T11:18:08.000Z,POPSICLE YOGURT\n#masakgapakeribet\n\nSuper ea...,"[#masakgapakeribet, #kidsactivity, #newnormal,...",@cimoryindonesia
9,https://www.instagram.com/p/CDAlVG1pKGS/,yackikuka,video,"137,325",2w,"Aug 7, 2020",2020-08-07T13:15:30.000Z,MPASI Kreasi Ubi Ungu untuk Adek Bayi\n\nHaii ...,"[#KebaikanSUN, #homemade, #biskuitbayi, #babyr...",@KasihSUN


In [14]:
all_post6 = all_post[250:]

all_post_details6 = [insta_link_details(url) for url in all_post6]
food_dataframe6 = pd.DataFrame(all_post_details6)
food_dataframe6

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CDqYAXThcc3/,devispantry,photo,68,AUGUST 9,"Aug 9, 2020",2020-08-09T08:37:31.000Z,PAKAI SARDEN! Resep Rica-Rica Khas Manado [Kua...,"[#ResepSarden, #ResepSardenRicaRica, #ResepDev...",
1,https://www.instagram.com/p/CDlRgkIB29E/,devispantry,photo,65,AUGUST 7,"Aug 7, 2020",2020-08-07T09:04:33.000Z,"LIVE NOW! Jumat, 7 Agustus jam 16:00 di @erasp...","[#MakinMudah, #eraspaceid, #extraordinaryexper...","[@eraspaceid, @endeus, @eraspaceid]"
2,https://www.instagram.com/p/CDfb2lvB0fP/,devispantry,photo,98,2w,"Aug 6, 2020",2020-08-06T07:17:01.000Z,"Jumat, 7 Agustus jam 12:00. Bingung masak apa ...","[#ivenet, #ivenetid, #ivenetkids, #koreanbabys...",
3,https://www.instagram.com/p/CDdf76Eh7Lz/,devispantry,photo,88,2w,"Aug 4, 2020",2020-08-04T08:39:51.000Z,Masih banyak stok daging di kulkas? Bikin sate...,"[#resep, #reseprumahan, #resepnusantara, #rese...",@vivacoid
4,https://www.instagram.com/p/CDapBL1B1LN/,devispantry,photo,269,2w,"Aug 3, 2020",2020-08-03T06:12:46.000Z,"Perhatiin bagian tengahnya, mantep bangettt. C...",,@dektih
...,...,...,...,...,...,...,...,...,...,...
105,https://www.instagram.com/p/CBAiMQjBOej/,xanderskitchen,photo,"5,531",11w,"Jun 5, 2020",2020-06-05T00:15:01.000Z,Soree .... bikin mie godog nyemek2 ...mayan en...,#xanderskitchenrecipe,
106,https://www.instagram.com/p/CAmeqGVhLRN/,xanderskitchen,photo,"2,840",12w,"May 26, 2020",2020-05-26T01:34:40.000Z,".\n.\nKemarin saya tidur cepat , di WA jam 8 m...",,@xanderskitchen
107,https://www.instagram.com/p/CAjs0d6hz0S/,xanderskitchen,photo,"8,932",12w,"May 25, 2020",2020-05-25T14:15:06.000Z,.\n.\nSelamat Hari Raya Idul Fitri . Mohon maa...,,
108,https://www.instagram.com/p/CAh3pnGBLjB/,xanderskitchen,video,"38,251",12w,"May 23, 2020",2020-05-23T13:18:59.000Z,.\n.\nMohon maaf lahir dan batin ya teman2 🙏🙏🙏...,,


In [15]:
all_dataset = pd.DataFrame()
all_dataset = all_dataset.append(food_dataframe1, ignore_index=True)
all_dataset = all_dataset.append(food_dataframe2, ignore_index=True)
all_dataset = all_dataset.append(food_dataframe3, ignore_index=True)
all_dataset = all_dataset.append(food_dataframe4, ignore_index=True)
all_dataset = all_dataset.append(food_dataframe5, ignore_index=True)
all_dataset = all_dataset.append(food_dataframe6, ignore_index=True)
all_dataset

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CELL_kAAdb_/,henjiwong,video,"54,958",1h,"Aug 22, 2020",2020-08-22T03:26:10.000Z,DESSERT BOX BY NAJLA CADBURY\n-\n-\nYukkk biki...,,
1,https://www.instagram.com/p/CEI8cMagKOb/,henjiwong,video,"16,651",22h,"Aug 21, 2020",2020-08-21T06:38:46.000Z,PANDAN SERIES!!\n-\n-\nWah yang suka wangi pan...,"[#BobaSatuIndonesia, #Tablesituation, #ontheta...",@bube
2,https://www.instagram.com/p/CEInTOXgGak/,henjiwong,video,"61,829",23h,"Aug 21, 2020",2020-08-21T05:28:04.000Z,MOCHI ANAK KOS\n-\n-\nSbnarnya mau kasi resep ...,"[#Tablesituation, #onthetableproject, #lovetoe...",
3,https://www.instagram.com/p/CEGCeoalCmp/,henjiwong,video,"233,467",13h,"Aug 21, 2020",2020-08-21T14:58:49.000Z,Japanese Souffle Cheesecake TANPA Cream Cheese...,,
4,https://www.instagram.com/p/CEEKTLcgbrk/,henjiwong,video,"47,487",2d,"Aug 19, 2020",2020-08-19T14:50:17.000Z,CEMILAN SEHAT DULU YUKK!!\n-\n-\nKali ini kita...,,@infosuperindo
...,...,...,...,...,...,...,...,...,...,...
355,https://www.instagram.com/p/CBAiMQjBOej/,xanderskitchen,photo,"5,531",11w,"Jun 5, 2020",2020-06-05T00:15:01.000Z,Soree .... bikin mie godog nyemek2 ...mayan en...,#xanderskitchenrecipe,
356,https://www.instagram.com/p/CAmeqGVhLRN/,xanderskitchen,photo,"2,840",12w,"May 26, 2020",2020-05-26T01:34:40.000Z,".\n.\nKemarin saya tidur cepat , di WA jam 8 m...",,@xanderskitchen
357,https://www.instagram.com/p/CAjs0d6hz0S/,xanderskitchen,photo,"8,932",12w,"May 25, 2020",2020-05-25T14:15:06.000Z,.\n.\nSelamat Hari Raya Idul Fitri . Mohon maa...,,
358,https://www.instagram.com/p/CAh3pnGBLjB/,xanderskitchen,video,"38,251",12w,"May 23, 2020",2020-05-23T13:18:59.000Z,.\n.\nMohon maaf lahir dan batin ya teman2 🙏🙏🙏...,,


In [16]:
all_dataset.to_csv('Downloads/influencer_list.csv', index=False)

In [17]:
# username='henjiwong'
# browser = webdriver.Chrome(ChromeDriverManager().install())
# browser.get('https://www.instagram.com/'+username+'/?hl=en')
# Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [18]:
# #Extract links from user profile page
# links=[]
# source = browser.page_source
# data = bs(source, 'html.parser')
# body = data.find('body')
# script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
# page_json = script.string.split(' = ', 1)[1].rstrip(';')
# data = json.loads(page_json)


# for link in data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']:
#     links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')

In [19]:
# data

In [20]:
# links

In [21]:
# body

In [22]:
# metrics = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['edges']

In [23]:
# result = []
# for node in metrics:
#     node = node.get('node')
#     if node and isinstance(node, dict):
#         result.append(node)

In [24]:
# result

In [25]:
# df = pd.DataFrame(result, columns =['__typename','id','shortcode','dimensions','display_url','gating_info', 'fact_check_overall_rating','fact_check_information','media_preview','owner','is_video','accessibility_caption','edge_media_to_caption','edge_media_to_comment','comments_disabled','taken_at_timestamp','edge_liked_by','edge_media_preview_like','location','thumbnail_src','thumbnail_resources']) 
# df

In [26]:
# caption_list = []

# for node in metrics:
#     caption = node.get('node').get('edge_media_to_caption')
#     if node and isinstance(node, dict):
#         caption_list.append(caption)

In [27]:
# caption_list

# Hashtag Page

In [28]:
# browser = webdriver.Chrome(ChromeDriverManager().install())
        
# browser.get('https://www.instagram.com/accounts/login/')

# emailInput = browser.find_elements_by_css_selector('form input')[0]
# passwordInput = browser.find_elements_by_css_selector('form input')[1]

# emailInput.send_keys('username')
# passwordInput.send_keys('password')
# passwordInput.send_keys(Keys.ENTER)
# time.sleep(2)

# hashtag='resep'
# browser.get('https://www.instagram.com/explore/tags/'+hashtag)
# Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")

In [29]:
browser = webdriver.Chrome(ChromeDriverManager().install())
hashtag_list=['resep','resepmasakan','resepviral','anekaresep','doyanmasak']
all_hashtag_post = []

for hashtag in hashtag_list:
    browser.get('https://www.instagram.com/explore/tags/'+hashtag)
    Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    post = 'https://www.instagram.com/p/'
    post_links = []
    while len(post_links) < 200:
        links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
        for link in links:
            if post in link and link not in post_links:
                post_links.append(link)
                all_hashtag_post.append(link)
        scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
        browser.execute_script(scroll_down)
        time.sleep(3)
    else:
        post_links[:200]

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/oliviatanuwidjaja/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [30]:
hashtag1 = all_hashtag_post[:1]
hashtag1

['https://www.instagram.com/p/CEHAs-Sh9jn/']

In [31]:
browser = webdriver.Chrome(ChromeDriverManager().install())
hashtag1 = all_hashtag_post[:100]

recipe_dataframe = pd.DataFrame()
for url in hashtag1:
    try:
        recipe_dataframe1 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe1, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/oliviatanuwidjaja/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
501,https://www.instagram.com/p/CELebMlpMCU/,celiarayha,photo,6,1 HOUR AGO,"Aug 22, 2020",2020-08-22T05:08:33.000Z,"Masakan yg sering dimasak ibu, favoritx bapak ...",#masakanrumah,
502,https://www.instagram.com/p/CELebMlpMCU/,celiarayha,photo,6,1 HOUR AGO,"Aug 22, 2020",2020-08-22T05:08:33.000Z,"Masakan yg sering dimasak ibu, favoritx bapak ...",#homemadefood,
503,https://www.instagram.com/p/CELebMlpMCU/,celiarayha,photo,6,1 HOUR AGO,"Aug 22, 2020",2020-08-22T05:08:33.000Z,"Masakan yg sering dimasak ibu, favoritx bapak ...",#supermoto,
504,https://www.instagram.com/p/CELebMlpMCU/,celiarayha,photo,6,1 HOUR AGO,"Aug 22, 2020",2020-08-22T05:08:33.000Z,"Masakan yg sering dimasak ibu, favoritx bapak ...",#instahits,


In [32]:
recipe_dataframe['link'].nunique()

36

In [33]:
hashtag2 = all_hashtag_post[100:200]

for url in hashtag2:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
1199,https://www.instagram.com/p/CELZCjqjzCO/,symbolhaty,photo,3,2 HOURS AGO,"Aug 22, 2020",2020-08-22T04:21:30.000Z,"Onion Rings\nRp. 8.000,-\nMlungker mlungker ka...",#enak,
1200,https://www.instagram.com/p/CELZCjqjzCO/,symbolhaty,photo,3,2 HOURS AGO,"Aug 22, 2020",2020-08-22T04:21:30.000Z,"Onion Rings\nRp. 8.000,-\nMlungker mlungker ka...",#dukuhankendal,
1201,https://www.instagram.com/p/CEK7OXRgdxq/,tutut.megawati,photo,8,2h,"Aug 22, 2020",2020-08-22T04:24:18.000Z,Weekend dirumah aja 😁\n\nDari pada tiduran doa...,,@tututmegaaa
1202,https://www.instagram.com/p/CEK7OXRgdxq/,tutut.megawati,photo,8,2h,"Aug 22, 2020",2020-08-22T04:24:18.000Z,Weekend dirumah aja 😁\n\nDari pada tiduran doa...,,@tututmegaaa


In [34]:
hashtag3 = all_hashtag_post[200:300]

for url in hashtag3:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
1725,https://www.instagram.com/p/CELfQOoBum8/,resepmakanannusantara.tv,photo,23,1h,"Aug 22, 2020",2020-08-22T05:16:14.000Z,"🗣 Kak, Sentuh dl LOVE ❤️ nya ya & Tolong Kasih...",,@caramemasak
1726,https://www.instagram.com/p/CELfQOoBum8/,resepmakanannusantara.tv,photo,23,1h,"Aug 22, 2020",2020-08-22T05:16:14.000Z,"🗣 Kak, Sentuh dl LOVE ❤️ nya ya & Tolong Kasih...",,@ocha
1727,https://www.instagram.com/p/CELfQOoBum8/,resepmakanannusantara.tv,photo,23,1h,"Aug 22, 2020",2020-08-22T05:16:14.000Z,"🗣 Kak, Sentuh dl LOVE ❤️ nya ya & Tolong Kasih...",,@resepmasakan
1728,https://www.instagram.com/p/CELfQOoBum8/,resepmakanannusantara.tv,photo,23,1h,"Aug 22, 2020",2020-08-22T05:16:14.000Z,"🗣 Kak, Sentuh dl LOVE ❤️ nya ya & Tolong Kasih...",,@caramemasak


In [35]:
hashtag4 = all_hashtag_post[300:400]

for url in hashtag4:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
2395,https://www.instagram.com/p/CELaSpNJSxs/,recipe.tastyfoodie,video,9,2 HOURS AGO,"Aug 22, 2020",2020-08-22T04:33:03.000Z,.⁣⁣\nDendeng Balado⁣\n⁣⁣\nCredit Tiktok @abiga...,#doyanmasak,@abigailaudity
2396,https://www.instagram.com/p/CELaSpNJSxs/,recipe.tastyfoodie,video,9,2 HOURS AGO,"Aug 22, 2020",2020-08-22T04:33:03.000Z,.⁣⁣\nDendeng Balado⁣\n⁣⁣\nCredit Tiktok @abiga...,#hobimasak,@abigailaudity
2397,https://www.instagram.com/p/CELaSpNJSxs/,recipe.tastyfoodie,video,9,2 HOURS AGO,"Aug 22, 2020",2020-08-22T04:33:03.000Z,.⁣⁣\nDendeng Balado⁣\n⁣⁣\nCredit Tiktok @abiga...,#resepnusantaraberbagiresep,@abigailaudity
2398,https://www.instagram.com/p/CELaSpNJSxs/,recipe.tastyfoodie,video,9,2 HOURS AGO,"Aug 22, 2020",2020-08-22T04:33:03.000Z,.⁣⁣\nDendeng Balado⁣\n⁣⁣\nCredit Tiktok @abiga...,#resep,@abigailaudity


In [36]:
hashtag5 = all_hashtag_post[400:500]

for url in hashtag5:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
2862,https://www.instagram.com/p/CEGACFDD_Ah/,tepung.lokalloka,photo,5,2d,"Aug 20, 2020",2020-08-20T02:46:47.000Z,Bun.. tahu nggak ada yang datang mengendap-end...,#snacksehat,
2863,https://www.instagram.com/p/CEGACFDD_Ah/,tepung.lokalloka,photo,5,2d,"Aug 20, 2020",2020-08-20T02:46:47.000Z,Bun.. tahu nggak ada yang datang mengendap-end...,#kuebebasgluten,
2864,https://www.instagram.com/p/CEGACFDD_Ah/,tepung.lokalloka,photo,5,2d,"Aug 20, 2020",2020-08-20T02:46:47.000Z,Bun.. tahu nggak ada yang datang mengendap-end...,#glutenfreeindonesia,
2865,https://www.instagram.com/p/CEGACFDD_Ah/,tepung.lokalloka,photo,5,2d,"Aug 20, 2020",2020-08-20T02:46:47.000Z,Bun.. tahu nggak ada yang datang mengendap-end...,#camilanautisme,


In [37]:
hashtag6 = all_hashtag_post[500:600]

for url in hashtag6:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
3535,https://www.instagram.com/p/CD71ppqDCvj/,dapur.sibobon,video,565,6 DAYS AGO,"Aug 16, 2020",2020-08-16T03:27:39.000Z,Assalamualaikum..\nBebikinan apa hari ini tema...,#resepenak,@dapur
3536,https://www.instagram.com/p/CD71ppqDCvj/,dapur.sibobon,video,565,6 DAYS AGO,"Aug 16, 2020",2020-08-16T03:27:39.000Z,Assalamualaikum..\nBebikinan apa hari ini tema...,#resepsimplerumahan,@dapur
3537,https://www.instagram.com/p/CD71ppqDCvj/,dapur.sibobon,video,565,6 DAYS AGO,"Aug 16, 2020",2020-08-16T03:27:39.000Z,Assalamualaikum..\nBebikinan apa hari ini tema...,#reseprumahan,@dapur
3538,https://www.instagram.com/p/CD71ppqDCvj/,dapur.sibobon,video,565,6 DAYS AGO,"Aug 16, 2020",2020-08-16T03:27:39.000Z,Assalamualaikum..\nBebikinan apa hari ini tema...,#resepmudah,@dapur


In [38]:
hashtag7 = all_hashtag_post[600:700]

for url in hashtag7:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
4237,https://www.instagram.com/p/CELau1wgJQW/,cookma_nyus,video,67,3 HOURS AGO,"Aug 22, 2020",2020-08-22T04:37:01.000Z,".⁣\nCLASSIC TIRAMISU⁣\n[TANPA MASCARPONE, WHIP...",#resepsederhana,
4238,https://www.instagram.com/p/CELau1wgJQW/,cookma_nyus,video,67,3 HOURS AGO,"Aug 22, 2020",2020-08-22T04:37:01.000Z,".⁣\nCLASSIC TIRAMISU⁣\n[TANPA MASCARPONE, WHIP...",#anekaresep,
4239,https://www.instagram.com/p/CELau1wgJQW/,cookma_nyus,video,67,3 HOURS AGO,"Aug 22, 2020",2020-08-22T04:37:01.000Z,".⁣\nCLASSIC TIRAMISU⁣\n[TANPA MASCARPONE, WHIP...",#resepmasakanfavorit,
4240,https://www.instagram.com/p/CELau1wgJQW/,cookma_nyus,video,67,3 HOURS AGO,"Aug 22, 2020",2020-08-22T04:37:01.000Z,".⁣\nCLASSIC TIRAMISU⁣\n[TANPA MASCARPONE, WHIP...",#anekakulinerlezat,


In [39]:
hashtag8 = all_hashtag_post[700:]

for url in hashtag8:
    try:
        recipe_dataframe2 = pd.DataFrame(insta_link_details(url))
        recipe_dataframe = recipe_dataframe.append(recipe_dataframe2, ignore_index=True)
    except Exception:
        pass

recipe_dataframe

,link,username,type,likes_views,age,post_date,post_time,comment,hashtags,mentions
0,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@meily
1,https://www.instagram.com/p/CEHAs-Sh9jn/,ketagihanmasak,photo,"1,806",1d,"Aug 20, 2020",2020-08-20T11:32:02.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
2,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@umi
3,https://www.instagram.com/p/CELGjtSh_sO/,ketagihanmasak,photo,432,4h,"Aug 22, 2020",2020-08-22T01:40:11.000Z,"Sebelum baca resep, kasih LOVE dulu ya bund .....",,@ketagihanmasak
4,https://www.instagram.com/p/CELL23tjU1A/,kenyangmakan,photo,237,3h,"Aug 22, 2020",2020-08-22T02:26:52.000Z,:\nSederhana tapi nikmat. Walaupun cuma telor ...,#sambal,@aras
...,...,...,...,...,...,...,...,...,...,...
5933,https://www.instagram.com/p/CELRM5zjr-s/,resep_kue_hits,photo,15,5 HOURS AGO,"Aug 22, 2020",2020-08-22T03:13:00.000Z,JANGAN LUPA FOLLOW DAN TAB LOVE 💗💗💗 YAAA...😍😍⁣...,#guniezzt,@fenita
5934,https://www.instagram.com/p/CELRM5zjr-s/,resep_kue_hits,photo,15,5 HOURS AGO,"Aug 22, 2020",2020-08-22T03:13:00.000Z,JANGAN LUPA FOLLOW DAN TAB LOVE 💗💗💗 YAAA...😍😍⁣...,#kuetradisional,@fenita
5935,https://www.instagram.com/p/CELRM5zjr-s/,resep_kue_hits,photo,15,5 HOURS AGO,"Aug 22, 2020",2020-08-22T03:13:00.000Z,JANGAN LUPA FOLLOW DAN TAB LOVE 💗💗💗 YAAA...😍😍⁣...,#doyanbakingcemilan,@fenita
5936,https://www.instagram.com/p/CELRM5zjr-s/,resep_kue_hits,photo,15,5 HOURS AGO,"Aug 22, 2020",2020-08-22T03:13:00.000Z,JANGAN LUPA FOLLOW DAN TAB LOVE 💗💗💗 YAAA...😍😍⁣...,#masakannusantara,@fenita


In [40]:
recipe_dataframe.to_csv('Downloads/recipe_list.csv', index=False)

In [77]:
# hashtag='resep'
# browser = webdriver.Chrome(ChromeDriverManager().install())
# browser.get('https://www.instagram.com/explore/tags/'+hashtag)
# Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")

In [78]:
# #Extract links from hashtag page
# links=[]
# source = browser.page_source
# data=bs(source, 'html.parser')
# body = data.find('body')
# script = body.find('script', text=lambda t: t.startswith('window._sharedData'))
# page_json = script.string.split(' = ', 1)[1].rstrip(';')
# data = json.loads(page_json)

# for link in data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
#     links.append('https://www.instagram.com'+'/p/'+link['node']['shortcode']+'/')

In [79]:
# len(links)

In [80]:
# metrics = data['entry_data']['TagPage'][0]['graphql']['hashtag']['edge_hashtag_to_media']['edges']

In [81]:
# result = []
# for node in metrics:
#     node = node.get('node')
#     if node and isinstance(node, dict):
#         result.append(node)

In [82]:
# df = pd.DataFrame(result, columns =['__typename','id','shortcode','dimensions','display_url','gating_info', 'fact_check_overall_rating','fact_check_information','media_preview','owner','is_video','accessibility_caption','edge_media_to_caption','edge_media_to_comment','comments_disabled','taken_at_timestamp','edge_liked_by','edge_media_preview_like','location','thumbnail_src','thumbnail_resources']) 
# df

In [83]:
# df['accessibility_caption']